In [1]:
import pandas as pd
import ast
from collections import Counter

In [2]:
df = pd.read_parquet("data/0000.parquet", engine="pyarrow")
df['annotation'] = df['annotation'].apply(ast.literal_eval)

In [3]:
def prepro_annotation(labels:list):
    dic_list = [dic['emotion'] for dic in labels]
    return dic_list

def has_majority_first_value(pairs, min_count=3):
    first_values = [pair[0] for pair in pairs]
    counts = Counter(first_values)
    return any(cnt >= min_count for cnt in counts.values())

def has_majority_second_value(pairs, min_count=3):
    second_values = [pair[1] for pair in pairs]
    counts = Counter(second_values)
    return any(cnt >= min_count for cnt in counts.values())

def majority_value(values, min_count=3):
    counts = Counter(values)
    most_common_val, cnt = counts.most_common(1)[0]  # 가장 많이 나온 값
    if cnt >= min_count:
        return most_common_val
    return None

def max_value(lables:list):
    counts = Counter(lables)
    return counts.most_common(1)[0][0]

### 데이터 중분류, 소분류 확인

In [4]:
df['annotation']= df['annotation'].apply(prepro_annotation)

In [5]:
df["annotations_split"] = df["annotation"].apply(lambda x: [item.split("_") for item in x])

In [6]:
annotations_list = [pair for sublist in df['annotations_split'] for pair in sublist]

In [7]:
label1 = [label[0] for label in annotations_list]
label2 = [label[1] for label in annotations_list]

In [8]:
print(set(label1))

{'두려움', '사랑', '미움(상대방)', '슬픔', '수치심', '싫어함(상태)', '중립', '분노', '기쁨', '욕망'}


In [9]:
print(set(label2))

{'위축감', '불쾌', '통쾌함', '발열', '공감', '갈등', '호감', '안정감', '아픔', '놀람', '반감', '경멸', '아른거림', '귀중함', '타오름', '서먹함', '심심함', '궁금함', '고통', '냉담', '너그러움', '편안함', '열정적인', '외면', '자신감', '허망', '후회', '원망', '반가움', '날카로움', '두근거림', '난처함', '동정(슬픔)', '부끄러움', '미안함', '외로움', '절망', '고마움', '실망', '불신감', '싫증', '신뢰감', '치사함', '공포', '사나움', '답답함', '초조함', '억울함', '불만', '비위상함', '그리움', '아쉬움', '매력적', '감동', '다정함', '시기심', '기대감', '걱정', '무기력', '수치심', '죄책감', '불편함', '즐거움', '신명남', '자랑스러움', '만족감', '욕심'}


In [10]:
print(len(set(label1)))
print(len(set(label2)))

10
67


### 동일 중분류 3개 이상인 데이터

In [ ]:
df_filtered

In [11]:
df.head()

,created_date,source,context,annotation,__index_level_0__,annotations_split
0,2024-02-03,X,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,"[기쁨_만족감, 기쁨_만족감, 기쁨_감동, 기쁨_감동, 기쁨_만족감]",0,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 감동], [기쁨, 감동], [기쁨..."
1,2024-02-07,X,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",1,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [..."
2,2024-02-08,X,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",2,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [..."
3,2024-02-09,X,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,"[슬픔_무기력, 싫어함(상태)_무기력, 슬픔_무기력, 슬픔_무기력, 슬픔_무기력]",3,"[[슬픔, 무기력], [싫어함(상태), 무기력], [슬픔, 무기력], [슬픔, 무기..."
4,2024-02-10,X,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,"[기쁨_즐거움, 기쁨_통쾌함, 기쁨_통쾌함, 기쁨_즐거움, 기쁨_즐거움]",4,"[[기쁨, 즐거움], [기쁨, 통쾌함], [기쁨, 통쾌함], [기쁨, 즐거움], [..."


In [12]:
df_filtered = df[df['annotations_split'].apply(has_majority_first_value)].reset_index()

In [13]:
df_filtered = df_filtered[df_filtered['annotations_split'].apply(has_majority_second_value)].reset_index()

In [14]:
df_filtered

,level_0,index,created_date,source,context,annotation,__index_level_0__,annotations_split
0,0,0,2024-02-03,X,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,"[기쁨_만족감, 기쁨_만족감, 기쁨_감동, 기쁨_감동, 기쁨_만족감]",0,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 감동], [기쁨, 감동], [기쁨..."
1,1,1,2024-02-07,X,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",1,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [..."
2,2,2,2024-02-08,X,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",2,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [..."
3,3,3,2024-02-09,X,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,"[슬픔_무기력, 싫어함(상태)_무기력, 슬픔_무기력, 슬픔_무기력, 슬픔_무기력]",3,"[[슬픔, 무기력], [싫어함(상태), 무기력], [슬픔, 무기력], [슬픔, 무기..."
4,4,4,2024-02-10,X,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,"[기쁨_즐거움, 기쁨_통쾌함, 기쁨_통쾌함, 기쁨_즐거움, 기쁨_즐거움]",4,"[[기쁨, 즐거움], [기쁨, 통쾌함], [기쁨, 통쾌함], [기쁨, 즐거움], [..."
...,...,...,...,...,...,...,...,...
684,926,1030,2024-02-21,youtube,백종원 코스프레하는거 어이없음 치킨집만 한 사람이 무슨 방송에 나와서 요식업 대가인...,"[미움(상대방)_비위상함, 미움(상대방)_경멸, 미움(상대방)_반감, 미움(상대방)...",1030,"[[미움(상대방), 비위상함], [미움(상대방), 경멸], [미움(상대방), 반감]..."
685,928,1032,2024-02-21,youtube,아 진짜 유튜브에 지 존나 가난했다가 성공했다고 떠드는 것들 믿지 말라니까 그걸 못...,"[싫어함(상태)_답답함, 싫어함(상태)_답답함, 싫어함(상태)_답답함, 싫어함(상태...",1032,"[[싫어함(상태), 답답함], [싫어함(상태), 답답함], [싫어함(상태), 답답함..."
686,929,1033,2024-02-21,youtube,이렇게 좋아했던 컨텐츠과 유튜버가 또 가는구나..,"[슬픔_허망, 욕망_아쉬움, 슬픔_허망, 슬픔_허망, 욕망_아쉬움]",1033,"[[슬픔, 허망], [욕망, 아쉬움], [슬픔, 허망], [슬픔, 허망], [욕망,..."
687,930,1034,2024-02-21,youtube,축협이랑 감독도 문제인데 이강인은 할말이 없네 멱살 잡았다고 주먹을 날리냐..물론 ...,"[미움(상대방)_실망, 미움(상대방)_반감, 미움(상대방)_냉담, 미움(상대방)_반...",1034,"[[미움(상대방), 실망], [미움(상대방), 반감], [미움(상대방), 냉담], ..."


In [ ]:
not_filtered = df[~df['__index_level_0__'].isin(df_filtered['index'])]


,created_date,source,context,annotation,__index_level_0__,annotations_split
6,2024-02-11,X,"도가니 이후로 이렇게 분노만 가득 남은 영화는 오랜만,,,,,,,,,,진짜 뭐 하나...","[분노_발열, 분노_타오름, 분노_발열, 분노_사나움, 분노_타오름]",6,"[[분노, 발열], [분노, 타오름], [분노, 발열], [분노, 사나움], [분노..."
8,2024-02-12,X,나는 솔직히 미인대회도 소름이지만 이때 도파민 쳐돌았음 크루즈에서 크라임씬 최초 연...,"[기쁨_즐거움, 기쁨_놀람, 욕망_기대감, 두려움_놀람, 기쁨_즐거움]",8,"[[기쁨, 즐거움], [기쁨, 놀람], [욕망, 기대감], [두려움, 놀람], [기..."
9,2024-02-13,X,어쩌라는 건지 다 말해놓고 기분나쁘게 받아들이지말라어쩌고\n꿀밤 개씨게때려야함,"[분노_불쾌, 분노_날카로움, 분노_타오름, 미움(상대방)_치사함, 분노_불쾌]",9,"[[분노, 불쾌], [분노, 날카로움], [분노, 타오름], [미움(상대방), 치사..."
10,2024-02-13,X,사진찍은꼴봐라\n열받음,"[분노_발열, 분노_날카로움, 분노_원망, 분노_날카로움, 분노_타오름]",10,"[[분노, 발열], [분노, 날카로움], [분노, 원망], [분노, 날카로움], [..."
21,2024-02-15,X,개인적으로 황정민의 감정 과잉 연기는 다 비슷한 것 같음. 대머리 말고는 전두환이랑...,"[욕망_아쉬움, 싫어함(상태)_싫증, 욕망_아쉬움, 욕망_불만, 싫어함(상태)_불편함]",21,"[[욕망, 아쉬움], [싫어함(상태), 싫증], [욕망, 아쉬움], [욕망, 불만]..."
...,...,...,...,...,...,...
1020,2024-02-21,youtube,당연히 여기 올라올만한 괴인인데 여기 올라온다는게 너무 슬프다,"[슬픔_허망, 슬픔_아쉬움, 슬픔_허망, 슬픔_실망, 슬픔_수치심]",1020,"[[슬픔, 허망], [슬픔, 아쉬움], [슬픔, 허망], [슬픔, 실망], [슬픔,..."
1031,2024-02-21,youtube,세삼 20억 빛있는데상태로 장사시작해 성공한 장사의 신 백종원이 대단하게 느껴지네..,"[기쁨_자랑스러움, 중립_신뢰감, 사랑_매력적, 기쁨_공감, 기쁨_공감]",1031,"[[기쁨, 자랑스러움], [중립, 신뢰감], [사랑, 매력적], [기쁨, 공감], ..."
1035,2024-02-21,youtube,애초에 이게 진실이든 협회가 그냥 시선 돌리는거이든 협회는 뭔 생각으로 진실이라고 ...,"[미움(상대방)_반감, 미움(상대방)_불신감, 미움(상대방)_냉담, 미움(상대방)_...",1035,"[[미움(상대방), 반감], [미움(상대방), 불신감], [미움(상대방), 냉담],..."
1036,2024-02-21,youtube,문뜩 10년째 인성논란없이 쭉 정상에 서있는 페이커가 새삼 대단하게 느껴지네...,"[기쁨_자랑스러움, 사랑_호감, 기쁨_자랑스러움, 기쁨_감동, 사랑_감동]",1036,"[[기쁨, 자랑스러움], [사랑, 호감], [기쁨, 자랑스러움], [기쁨, 감동],..."


In [15]:
# 0번째 값
df_filtered['category1'] = df_filtered['annotations_split'].apply(lambda pairs: majority_value([p[0] for p in pairs]))

# 1번째 값
df_filtered['category2'] = df_filtered['annotations_split'].apply(lambda pairs: majority_value([p[1] for p in pairs]))

In [16]:
df_filtered

,level_0,index,created_date,source,context,annotation,__index_level_0__,annotations_split,category1,category2
0,0,0,2024-02-03,X,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,"[기쁨_만족감, 기쁨_만족감, 기쁨_감동, 기쁨_감동, 기쁨_만족감]",0,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 감동], [기쁨, 감동], [기쁨...",기쁨,만족감
1,1,1,2024-02-07,X,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",1,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [...",기쁨,만족감
2,2,2,2024-02-08,X,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",2,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [...",기쁨,만족감
3,3,3,2024-02-09,X,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,"[슬픔_무기력, 싫어함(상태)_무기력, 슬픔_무기력, 슬픔_무기력, 슬픔_무기력]",3,"[[슬픔, 무기력], [싫어함(상태), 무기력], [슬픔, 무기력], [슬픔, 무기...",슬픔,무기력
4,4,4,2024-02-10,X,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,"[기쁨_즐거움, 기쁨_통쾌함, 기쁨_통쾌함, 기쁨_즐거움, 기쁨_즐거움]",4,"[[기쁨, 즐거움], [기쁨, 통쾌함], [기쁨, 통쾌함], [기쁨, 즐거움], [...",기쁨,즐거움
...,...,...,...,...,...,...,...,...,...,...
684,926,1030,2024-02-21,youtube,백종원 코스프레하는거 어이없음 치킨집만 한 사람이 무슨 방송에 나와서 요식업 대가인...,"[미움(상대방)_비위상함, 미움(상대방)_경멸, 미움(상대방)_반감, 미움(상대방)...",1030,"[[미움(상대방), 비위상함], [미움(상대방), 경멸], [미움(상대방), 반감]...",미움(상대방),반감
685,928,1032,2024-02-21,youtube,아 진짜 유튜브에 지 존나 가난했다가 성공했다고 떠드는 것들 믿지 말라니까 그걸 못...,"[싫어함(상태)_답답함, 싫어함(상태)_답답함, 싫어함(상태)_답답함, 싫어함(상태...",1032,"[[싫어함(상태), 답답함], [싫어함(상태), 답답함], [싫어함(상태), 답답함...",싫어함(상태),답답함
686,929,1033,2024-02-21,youtube,이렇게 좋아했던 컨텐츠과 유튜버가 또 가는구나..,"[슬픔_허망, 욕망_아쉬움, 슬픔_허망, 슬픔_허망, 욕망_아쉬움]",1033,"[[슬픔, 허망], [욕망, 아쉬움], [슬픔, 허망], [슬픔, 허망], [욕망,...",슬픔,허망
687,930,1034,2024-02-21,youtube,축협이랑 감독도 문제인데 이강인은 할말이 없네 멱살 잡았다고 주먹을 날리냐..물론 ...,"[미움(상대방)_실망, 미움(상대방)_반감, 미움(상대방)_냉담, 미움(상대방)_반...",1034,"[[미움(상대방), 실망], [미움(상대방), 반감], [미움(상대방), 냉담], ...",미움(상대방),반감


In [17]:
df_filtered['category1'].value_counts()

category1
기쁨         188
슬픔         125
욕망          85
두려움         72
싫어함(상태)     53
사랑          52
미움(상대방)     44
분노          39
수치심         26
중립           5
Name: count, dtype: int64

In [18]:
print(df_filtered['category2'].value_counts())

category2
만족감       49
아쉬움       35
감동        34
놀람        32
동정(슬픔)    27
          ..
외면         1
자신감        1
갈등         1
신뢰감        1
통쾌함        1
Name: count, Length: 64, dtype: int64


In [19]:
# 제거해야 할 특정 조합들 정의
combinations_to_remove = {
    ('기쁨', '귀중함'),
    ('기쁨', '그리움'),
    ('기쁨', '아른거림'),
    ('기쁨', '호감'),
    ('두려움', '고통'),
    ('두려움', '궁금함'),
    ('두려움', '난처함'),
    ('두려움', '날카로움'),
    ('두려움', '만족감'),
    ('두려움', '부끄러움'),
    ('두려움', '불신감'),
    ('두려움', '불쾌'),
    ('두려움', '외로움'),
    ('두려움', '절망'),
    ('미움(상대방)', '날카로움'),
    ('미움(상대방)', '놀람'),
    ('미움(상대방)', '답답함'),
    ('미움(상대방)', '불만'),
    ('미움(상대방)', '불쾌'),
    ('미움(상대방)', '불편함'),
    ('미움(상대방)', '실망'),
    ('미움(상대방)', '아픔'),
    ('미움(상대방)', '원망'),
    ('미움(상대방)', '후회'),
    ('분노', '감동'),
    ('분노', '걱정'),
    ('분노', '경멸'),
    ('분노', '고통'),
    ('분노', '공감'),
    ('분노', '놀람'),
    ('분노', '답답함'),
    ('분노', '반감'),
    ('분노', '불만'),
    ('분노', '불신감'),
    ('분노', '비위상함'),
    ('분노', '시기심'),
    ('분노', '실망'),
    ('분노', '싫증'),
    ('분노', '억울함'),
    ('분노', '절망'),
    ('사랑', '감동'),
    ('사랑', '걱정'),
    ('사랑', '고마움'),
    ('사랑', '공감'),
    ('사랑', '그리움'),
    ('사랑', '기대감'),
    ('사랑', '놀람'),
    ('사랑', '만족감'),
    ('사랑', '열정적인'),
    ('사랑', '즐거움'),
    ('사랑', '편안함'),
    ('수치심', '난처함'),
    ('수치심', '놀람'),
    ('수치심', '답답함'),
    ('슬픔', '감동'),
    ('슬픔', '걱정'),
    ('슬픔', '고마움'),
    ('슬픔', '공감'),
    ('슬픔', '공포'),
    ('슬픔', '난처함'),
    ('슬픔', '놀람'),
    ('슬픔', '답답함'),
    ('슬픔', '매력적'),
    ('슬픔', '미안함'),
    ('슬픔', '서먹함'),
    ('슬픔', '싫증'),
    ('슬픔', '아쉬움'),
    ('슬픔', '위축감'),
    ('슬픔', '자랑스러움'),
    ('싫어함(상태)', '걱정'),
    ('싫어함(상태)', '경멸'),
    ('싫어함(상태)', '고통'),
    ('싫어함(상태)', '궁금함'),
    ('싫어함(상태)', '냉담'),
    ('싫어함(상태)', '놀람'),
    ('싫어함(상태)', '무기력'),
    ('싫어함(상태)', '반감'),
    ('싫어함(상태)', '불만'),
    ('싫어함(상태)', '불신감'),
    ('싫어함(상태)', '불쾌'),
    ('싫어함(상태)', '비위상함'),
    ('싫어함(상태)', '실망'),
    ('싫어함(상태)', '아쉬움'),
    ('싫어함(상태)', '아픔'),
    ('싫어함(상태)', '억울함'),
    ('싫어함(상태)', '외면'),
    ('싫어함(상태)', '절망'),
    ('싫어함(상태)', '치사함'),
    ('싫어함(상태)', '후회'),
    ('욕망', '감동'),
    ('욕망', '걱정'),
    ('욕망', '난처함'),
    ('욕망', '놀람'),
    ('욕망', '답답함'),
    ('욕망', '만족감'),
    ('욕망', '무기력'),
    ('욕망', '시기심'),
    ('욕망', '실망'),
    ('욕망', '원망')
}

# 특정 조합만 제거하는 함수
def should_keep_row_specific(row):
    category = row['category1']
    subcategory = row['category2']
    
    # 제거 목록에 있는 조합이면 제거 (False 반환)
    return (category, subcategory) not in combinations_to_remove

# 특정 조합만 제거
df_anomaly = df_filtered[df_filtered.apply(should_keep_row_specific, axis=1)].reset_index(drop=True)

print(f"원본 데이터: {len(df_filtered)}개")
print(f"필터링 후: {len(df_anomaly)}개") 
print(f"제거된 데이터: {len(df_filtered) - len(df_anomaly)}개")

df_anomaly

원본 데이터: 689개
필터링 후: 664개
제거된 데이터: 25개


,level_0,index,created_date,source,context,annotation,__index_level_0__,annotations_split,category1,category2
0,0,0,2024-02-03,X,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,"[기쁨_만족감, 기쁨_만족감, 기쁨_감동, 기쁨_감동, 기쁨_만족감]",0,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 감동], [기쁨, 감동], [기쁨...",기쁨,만족감
1,1,1,2024-02-07,X,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",1,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [...",기쁨,만족감
2,2,2,2024-02-08,X,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,"[기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감, 기쁨_만족감]",2,"[[기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [기쁨, 만족감], [...",기쁨,만족감
3,3,3,2024-02-09,X,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,"[슬픔_무기력, 싫어함(상태)_무기력, 슬픔_무기력, 슬픔_무기력, 슬픔_무기력]",3,"[[슬픔, 무기력], [싫어함(상태), 무기력], [슬픔, 무기력], [슬픔, 무기...",슬픔,무기력
4,4,4,2024-02-10,X,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,"[기쁨_즐거움, 기쁨_통쾌함, 기쁨_통쾌함, 기쁨_즐거움, 기쁨_즐거움]",4,"[[기쁨, 즐거움], [기쁨, 통쾌함], [기쁨, 통쾌함], [기쁨, 즐거움], [...",기쁨,즐거움
...,...,...,...,...,...,...,...,...,...,...
659,926,1030,2024-02-21,youtube,백종원 코스프레하는거 어이없음 치킨집만 한 사람이 무슨 방송에 나와서 요식업 대가인...,"[미움(상대방)_비위상함, 미움(상대방)_경멸, 미움(상대방)_반감, 미움(상대방)...",1030,"[[미움(상대방), 비위상함], [미움(상대방), 경멸], [미움(상대방), 반감]...",미움(상대방),반감
660,928,1032,2024-02-21,youtube,아 진짜 유튜브에 지 존나 가난했다가 성공했다고 떠드는 것들 믿지 말라니까 그걸 못...,"[싫어함(상태)_답답함, 싫어함(상태)_답답함, 싫어함(상태)_답답함, 싫어함(상태...",1032,"[[싫어함(상태), 답답함], [싫어함(상태), 답답함], [싫어함(상태), 답답함...",싫어함(상태),답답함
661,929,1033,2024-02-21,youtube,이렇게 좋아했던 컨텐츠과 유튜버가 또 가는구나..,"[슬픔_허망, 욕망_아쉬움, 슬픔_허망, 슬픔_허망, 욕망_아쉬움]",1033,"[[슬픔, 허망], [욕망, 아쉬움], [슬픔, 허망], [슬픔, 허망], [욕망,...",슬픔,허망
662,930,1034,2024-02-21,youtube,축협이랑 감독도 문제인데 이강인은 할말이 없네 멱살 잡았다고 주먹을 날리냐..물론 ...,"[미움(상대방)_실망, 미움(상대방)_반감, 미움(상대방)_냉담, 미움(상대방)_반...",1034,"[[미움(상대방), 실망], [미움(상대방), 반감], [미움(상대방), 냉담], ...",미움(상대방),반감


In [20]:
result = df_anomaly[['index','context','annotations_split', 'category1', 'category2']]
result.to_excel('증강할데이터33.xlsx', index=False)